In [1]:
import os
import sys
import json
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from tqdm import tqdm

import torch.nn as nn
import torch

In [2]:
train_dir = r'/content/drive/MyDrive/cars_tanks/train'
test_dir = r'/content/drive/MyDrive/cars_tanks/test'
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channel, out_channel, stride=1, downsample=None, **kwargs):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel,
                               kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out

In [4]:
class Bottleneck(nn.Module):

    expansion = 4

    def __init__(self, in_channel, out_channel, stride=1, downsample=None,
                 groups=1, width_per_group=64):
        super(Bottleneck, self).__init__()

        width = int(out_channel * (width_per_group / 64.)) * groups

        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=width,
                               kernel_size=1, stride=1, bias=False)  # squeeze channels
        self.bn1 = nn.BatchNorm2d(width)
        # -----------------------------------------
        self.conv2 = nn.Conv2d(in_channels=width, out_channels=width, groups=groups,
                               kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(width)
        # -----------------------------------------
        self.conv3 = nn.Conv2d(in_channels=width, out_channels=out_channel*self.expansion,
                               kernel_size=1, stride=1, bias=False)  # unsqueeze channels
        self.bn3 = nn.BatchNorm2d(out_channel*self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out += identity
        out = self.relu(out)

        return out

In [5]:

class MyLeNet5(nn.Module):
    # 子类继承中重新定义Module类的__init__()和forward()函数
    # init()函数：进行初始化，申明模型中各层的定义
    def __init__(self):
        # super：引入父类的初始化方法给子类进行初始化
        super(MyLeNet5, self).__init__()
        # 卷积层，输入大小为28*28，输出大小为28*28，输入通道为1，输出为6，卷积核为5，扩充边缘为2
        self.c1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2)
        # 使用sigmoid作为激活函数
        self.Sigmoid = nn.Sigmoid()
        # AvgPool2d：二维平均池化操作
        # 池化层，输入大小为28*28，输出大小为14*14，输入通道为6，输出为6，卷积核为2，步长为2
        self.s2 = nn.AvgPool2d(kernel_size=2, stride=2)
        # 卷积层，输入大小为14*14，输出大小为10*10，输入通道为6，输出为16，卷积核为5
        self.c3 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        # 池化层，输入大小为10*10，输出大小为5*5，输入通道为16，输出为16，卷积核为2，步长为2
        self.s4 = nn.AvgPool2d(kernel_size=2, stride=2)
        # 卷积层，输入大小为5*5，输出大小为1*1，输入通道为16，输出为120，卷积核为5
        self.c5 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        # Flatten()：将张量（多维数组）平坦化处理，张量的第0维表示的是batch_size（数量），所以Flatten()默认从第二维开始平坦化
        self.flatten = nn.Flatten()
        # 全连接层
        # Linear（in_features，out_features）
        # in_features指的是[batch_size, size]中的size,即样本的大小
        # out_features指的是[batch_size，output_size]中的output_size，样本输出的维度大小，也代表了该全连接层的神经元个数
        self.f6 = nn.Linear(120, 84)
        # 全连接层&输出层
        self.output = nn.Linear(84, 10)

    # forward()：定义前向传播过程,描述了各层之间的连接关系
    def forward(self, x):
        # x输入为28*28*1， 输出为28*28*6
        x = self.Sigmoid(self.c1(x))
        # x输入为28*28*6，输出为14*14*6
        x = self.s2(x)
        # x输入为14*14*6，输出为10*10*16
        x = self.Sigmoid(self.c3(x))
        # x输入为10*10*16，输出为5*5*16
        x = self.s4(x)
        # x输入为5*5*16，输出为1*1*120
        x = self.c5(x)
        x = self.flatten(x)
        # x输入为120，输出为84
        x = self.f6(x)
        # x输入为84，输出为10
        x = self.output(x)
        return x

In [6]:
class tankCarCls(nn.Module):

    def __init__(self,
                 block,
                 blocks_num,
                 num_classes=1000,
                 include_top=True,
                 groups=1,
                 width_per_group=64):
        super(tankCarCls, self).__init__()
        self.include_top = include_top
        self.in_channel = 64

        self.groups = groups
        self.width_per_group = width_per_group

        self.conv1 = nn.Conv2d(3, self.in_channel, kernel_size=7, stride=2,
                               padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channel)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, blocks_num[0])
        self.layer2 = self._make_layer(block, 128, blocks_num[1], stride=2)
        self.layer3 = self._make_layer(block, 256, blocks_num[2], stride=2)
        self.layer4 = self._make_layer(block, 512, blocks_num[3], stride=2)
        if self.include_top:
            self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # output size = (1, 1)
            self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    def _make_layer(self, block, channel, block_num, stride=1):
        downsample = None
        if stride != 1 or self.in_channel != channel * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channel, channel * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(channel * block.expansion))

        layers = []
        layers.append(block(self.in_channel,
                            channel,
                            downsample=downsample,
                            stride=stride,
                            groups=self.groups,
                            width_per_group=self.width_per_group))
        self.in_channel = channel * block.expansion

        for _ in range(1, block_num):
            layers.append(block(self.in_channel,
                                channel,
                                groups=self.groups,
                                width_per_group=self.width_per_group))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)


        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        if self.include_top:
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
            x = self.fc(x)
        return x

In [7]:
def tankCarCls_v0(num_classes=2, include_top=True):
    return tankCarCls(Bottleneck, [3, 4, 6, 3], num_classes=num_classes, include_top=include_top)

In [8]:
def main():
    import os
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # 下面老是报错 shape 不一致

    #device = torch.device("cuda:0")
    device = torch.device("cpu")
    print("using {} device.".format(device))

    data_transform = {
        "train": transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
        "val": transforms.Compose([transforms.Resize(256),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

    data_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))  # get data root path
    train_dataset = datasets.ImageFolder(root=os.path.join("/content/drive/MyDrive/cars_tanks/train"),
                                         transform=data_transform["train"])
    train_num = len(train_dataset)


    flower_list = train_dataset.class_to_idx
    cla_dict = dict((val, key) for key, val in flower_list.items())
    # write dict into json file
    json_str = json.dumps(cla_dict, indent=2)
    with open('class_indices.json', 'w') as json_file:
        json_file.write(json_str)

    batch_size = 4
    nw = 0  # number of workers
    print('Using {} dataloader workers every process'.format(nw))

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size, shuffle=True,
                                               num_workers=nw)

    validate_dataset = datasets.ImageFolder(root=os.path.join("/content/drive/MyDrive/cars_tanks/train"),
                                            transform=data_transform["val"])
    val_num = len(validate_dataset)
    validate_loader = torch.utils.data.DataLoader(validate_dataset,
                                                  batch_size=batch_size, shuffle=False,
                                                  num_workers=nw)

    print("using {} images for training, {} images for validation.".format(train_num,
                                                                           val_num))

    net = tankCarCls_v0()

    # change fc layer structure
    in_channel = net.fc.in_features
    net.fc = nn.Linear(in_channel, 2)
    net.to(device)

    # define loss function
    loss_function = nn.CrossEntropyLoss()

    # construct an optimizer
    params = [p for p in net.parameters() if p.requires_grad]
    optimizer = optim.Adam(params, lr=0.0001)

    epochs = 30
    best_acc = 0.0
    save_path = './tankCarCls_v1.pth'
    train_steps = len(train_loader)
    torch.cuda.empty_cache()
    writer = SummaryWriter("logs")
    for epoch in range(epochs):
        # train
        net.train()
        running_loss = 0.0
        train_bar = tqdm(train_loader, file=sys.stdout)
        for step, data in enumerate(train_bar):
            images, labels = data
            optimizer.zero_grad()
            logits = net(images.to(device))
            loss = loss_function(logits, labels.to(device))
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,
                                                                     epochs,
                                                                     loss)

        # validate
        net.eval()
        acc = 0.0  # accumulate accurate number / epoch
        TP = 0.0
        FP = 0.0
        FN = 0.0
        TN = 0.0

        with torch.no_grad():
            val_bar = tqdm(validate_loader, file=sys.stdout)
            for val_data in val_bar:
                val_images, val_labels = val_data
                outputs = net(val_images.to(device))
                # loss = loss_function(outputs, test_labels)
                predict_y = torch.max(outputs, dim=1)[1]


                acc += torch.eq(predict_y, val_labels.to(device)).sum().item()


                TP += (torch.eq(predict_y, val_labels.to(device)) & torch.eq(predict_y, 0)).sum().item()

                # 10
                FP += (torch.eq(predict_y, 1) & torch.eq(val_labels.to(device), 0)).sum().item()
                # 01
                FN += (torch.eq(predict_y, 0) & torch.eq(val_labels.to(device), 1)).sum().item()
                # 11
                TN += (torch.eq(predict_y, 1) & torch.eq(val_labels.to(device), 1)).sum().item()

                val_bar.desc = "valid epoch[{}/{}]".format(epoch + 1,
                                                           epochs)

        val_accurate = acc / val_num  #（TP+TN）/（TP+FP+TN+FN）
        # Precision=TP/（TP+FP）
        Precision = TP /(TP + FP)
        Recall = TP / (TP + FN)
        # Precision-recall
        # Confusion matrix
        F1 = 2 * (Precision * Recall) / (Precision + Recall)
        Sensitivity = TP / (TP + FN)
        Specificity = TN / (TN + FP)
        print('[epoch %d] loss: %.3f  accuracy: %.3f  Precision: %.3f  Recall: %.3f  F1: %3f  Sensitivity: %.3f  Specificity: %.3f'   %
              (epoch + 1, running_loss / train_steps, val_accurate,Precision,Recall, F1, Sensitivity, Specificity))

        writer.add_scalar("Accuracy-epoch", val_accurate, epoch+1)
        writer.add_scalar("loss-epoch", running_loss, epoch+1)
        writer.add_scalar("Precision-epoch", Precision, epoch+1)
        writer.add_scalar("Recall-epoch", Recall, epoch+1)
        writer.add_scalar("Precision-Recall", Precision, Recall)
        writer.add_scalar("F1-epoch", F1, epoch + 1)
        writer.add_scalar("Sensitivity-epoch", Sensitivity, epoch + 1)
        writer.add_scalar("Specificity-epoch", Specificity, epoch + 1)
        writer.add_scalar("Sensitivity-Specificity", Sensitivity, Specificity)



        if val_accurate > best_acc:
            best_acc = val_accurate
            torch.save(net.state_dict(), save_path)
    writer.close()
    print('Finished Training')

In [9]:
if __name__ == '__main__':
    main()


using cpu device.
Using 0 dataloader workers every process
using 1303 images for training, 1303 images for validation.
valid epoch[1/30]: 100%|██████████| 326/326 [04:27<00:00,  1.22it/s]
[epoch 1] loss: 0.666  accuracy: 0.483  Precision: 0.107  Recall: 0.814  F1: 0.189904  Sensitivity: 0.814  Specificity: 0.456
train epoch[2/30] loss:0.584:  11%|█         | 35/326 [01:32<12:48,  2.64s/it]


KeyboardInterrupt: ignored